# Trying to use DuckDB for data import & cleansing
### TO DO remove pandas dependency by using duckdb python functions integration

## install packages

In [1]:
!pip install duckdb==0.8.1
!pip install Levenshtein==0.21.1
!pip install thefuzz==0.19.0
!pip install pyarrow==12.0.1
!pip install pandas==2.0.2

  Using cached duckdb-0.8.1-cp310-cp310-win_amd64.whl (9.8 MB)
  Using cached Levenshtein-0.21.1-cp310-cp310-win_amd64.whl (100 kB)
  Using cached rapidfuzz-3.1.1-cp310-cp310-win_amd64.whl (1.8 MB)
  Using cached thefuzz-0.19.0-py2.py3-none-any.whl (17 kB)
  Using cached pyarrow-12.0.1-cp310-cp310-win_amd64.whl (21.5 MB)
  Using cached numpy-1.25.1-cp310-cp310-win_amd64.whl (15.0 MB)
  Using cached pandas-2.0.2-cp310-cp310-win_amd64.whl (10.7 MB)
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)


In [2]:
import duckdb as db
import Levenshtein
from thefuzz import fuzz
import pandas as pd

In [3]:
TEXT_TO_EVAL = 'St. Albans'

## Data Import

In [4]:
#ugly hack to import data from csv - single quotes are not handled properly
sql = """
SELECT 
    trim(data,'[], ''') as text, false as target
FROM read_csv('negatives.txt', delim='\n,', quote='''', columns={'data': 'VARCHAR'})
union all
SELECT 
   trim(data,'[], ''') as text, true as target  
FROM read_csv('positives.txt', delim='\n,', columns={'data': 'VARCHAR'})
"""
data = db.query(sql).df()

## let's try with duckdb built-in similar to function

In [5]:
sql = f"""
SELECT *, text SIMILAR TO '{TEXT_TO_EVAL}' as similar_to
FROM data
"""
data = db.query(sql).df()

# calc accuracy
sql = """
    SELECT 
        sum(case when similar_to = target then 1 else 0 end)/count(*) * 100 as accuracy
    FROM 
        data
"""
db.query(sql)

┌───────────────────┐
│     accuracy      │
│      double       │
├───────────────────┤
│ 66.66666666666666 │
└───────────────────┘

### 66% is not good enough!

## Good old Levenshtein distance

In [6]:
#Calc Levenshtein distance over data set
data['levenshtein'] = data['text'].apply(lambda x: Levenshtein.distance(x, TEXT_TO_EVAL))

#calc Levenshtein accuracy
distance = [3]
sql = f"""
    SELECT 
        sum(case when (levenshtein <=3) = target then 1 else 0 end)/count(*) * 100 as accuracy
    FROM 
        data
"""
db.query(sql)

┌───────────────────┐
│     accuracy      │
│      double       │
├───────────────────┤
│ 97.61904761904762 │
└───────────────────┘

### 98% is better

## Trying TheFuzz library

In [7]:
data['fuzz_ratio'] = data['text'].apply(lambda x: fuzz.ratio(x, TEXT_TO_EVAL))
# calc accuracy
sql = f"""
    SELECT 
        sum(case when (fuzz_ratio >=80) = target then 1 else 0 end)/count(*) * 100 as accuracy
    FROM 
        data
"""
db.query(sql)


┌──────────┐
│ accuracy │
│  double  │
├──────────┤
│    100.0 │
└──────────┘

### 100% achieved